コード: https://github.com/yuki-inaho/dbscan_comparison

コミット: forkしたリポジトリのhotfix/memory_leaks_42e1e71ブランチの最新コミット(748040c67a182405523ae62a2647fc0cc257e7ff)

In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import os
import psutil

def get_memory_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss

In [23]:
import numpy as np
import dbscan
from dbscan import DBSCAN

print(dbscan.__version__)

# 初期のメモリ使用量を取得
initial_memory = get_memory_usage()
print(f"Initial memory usage: {initial_memory / 1024 / 1024:.2f} MB")

points = np.random.rand(100000, 3)
pcd_np = np.asarray(points)

labels_another, core_samples_mask = DBSCAN(pcd_np, eps=0.025, min_samples=10)

# DBSCAN実行後のメモリ使用量を取得
after_memory = get_memory_usage()
print(f"Memory usage after DBSCAN: {after_memory / 1024 / 1024:.2f} MB")

# メモリ使用量の差を計算
memory_diff = after_memory - initial_memory
print(f"Memory difference: {memory_diff / 1024 / 1024:.2f} MB")

0.0.12
Initial memory usage: 224.64 MB
Memory usage after DBSCAN: 224.86 MB
Memory difference: 0.22 MB


In [24]:
# 複数回DBSCANを実行し、メモリ使用量を確認
num_iterations = 15

for i in range(num_iterations):
    labels, core_samples = DBSCAN(pcd_np, eps=0.025, min_samples=10)
    current_memory = get_memory_usage()
    print(f"Iteration {i+1}: Memory usage: {current_memory / 1024 / 1024:.2f} MB")

Iteration 1: Memory usage: 224.84 MB
Iteration 2: Memory usage: 224.86 MB
Iteration 3: Memory usage: 224.87 MB
Iteration 4: Memory usage: 224.87 MB
Iteration 5: Memory usage: 224.87 MB
Iteration 6: Memory usage: 224.87 MB
Iteration 7: Memory usage: 224.87 MB
Iteration 8: Memory usage: 224.87 MB
Iteration 9: Memory usage: 224.87 MB
Iteration 10: Memory usage: 224.87 MB
Iteration 11: Memory usage: 224.86 MB
Iteration 12: Memory usage: 224.86 MB
Iteration 13: Memory usage: 224.86 MB
Iteration 14: Memory usage: 224.91 MB
Iteration 15: Memory usage: 224.90 MB


In [25]:
import os
import psutil
import numpy as np
from dbscan import DBSCAN
import sys
import weakref
import gc

# 初期のメモリ使用量を取得
def get_memory_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss

initial_memory = get_memory_usage()

# ランダムな点群データを生成
pcd_np = np.random.rand(100000, 3)

# DBSCANを実行
labels_another, core_samples_mask = DBSCAN(pcd_np, eps=0.025, min_samples=10)

# メモリ使用量と参照カウントを確認
print(f"Initial memory usage: {initial_memory / 1024 / 1024:.2f} MB")

# 参照カウントを確認
r = (labels_another, core_samples_mask)
print(f"Reference count of labels_another: {sys.getrefcount(labels_another)}")
print(f"Reference count of core_samples_mask: {sys.getrefcount(core_samples_mask)}")

# 弱参照を作成
weak_labels = weakref.ref(r[0])
weak_core_samples = weakref.ref(r[1])

# rへの参照を削除 + ガベージコレクションを実行
del r
del labels_another, core_samples_mask
gc.collect()

# 弱参照先のオブジェクトにアクセス
print(f"Weak reference to labels_another before second gc.collect(): {weak_labels()}")
print(f"Weak reference to core_samples_mask before second gc.collect(): {weak_core_samples()}")

# 再度ガベージコレクションを実行
gc.collect()

# 弱参照先のオブジェクトにアクセス
print(f"Weak reference to labels_another after second gc.collect(): {weak_labels()}")
print(f"Weak reference to core_samples_mask after second gc.collect(): {weak_core_samples()}")

# メモリ使用量の差を計算
memory_diff = current_memory - initial_memory
print(f"Memory difference after iteration {i+1}: {memory_diff / 1024 / 1024:.2f} MB")

# 追加のガベージコレクションの実行とメモリ使用量の確認
gc.collect()
current_memory = get_memory_usage()
print(f"Memory usage after additional gc.collect: {current_memory / 1024 / 1024:.2f} MB")


Initial memory usage: 224.90 MB
Reference count of labels_another: 3
Reference count of core_samples_mask: 3
Weak reference to labels_another before second gc.collect(): None
Weak reference to core_samples_mask before second gc.collect(): None
Weak reference to labels_another after second gc.collect(): None
Weak reference to core_samples_mask after second gc.collect(): None
Memory difference after iteration 15: 0.00 MB
Memory usage after additional gc.collect: 224.93 MB


- ガベージコレクション（gc.collect()）を実行した結果、弱参照g()がNoneを返せている
    - r[0]が参照するオブジェクトがガベージコレクション後に解放されている